In [1]:
import pandas as pd
import datetime
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from html_table_parser import parser_functions as parser

from IPython.display import clear_output

In [2]:
def loading_check(desc=''):
    while(True):
        
        time.sleep(1) 
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')    
        print("===> ", desc , " " , soup.select('#body > div.loadingMask.on'))
        
        if len(soup.select('#body > div.loadingMask.on')) == 0 :
            break        
        

In [3]:
def move_page(curr_page, last_page, btn_id, mode=None):
    # 메인화면 : sbGridPaging, 서브화면 : sbGridPaging2
    
    # print(f'!!! {curr_page}, {last_page}, {btn_id}')
    
    if curr_page == last_page:
        if btn_id == "sbGridPaging" and mode == "restart":
            print(f'{curr_page} 까지 이동을 완료하였습니다.')
        elif btn_id == "sbGridPaging":
            print("메인의 마지막 페이지 입니다.")
        elif btn_id == "sbGridPaging2":
            print("     ", curr_page, " 서브의 마지막 페이지 입니다.")  
    elif curr_page == 10 :
        # 10페이지일때 다음버튼 클릭
        # print("111 next : ", driver.find_element(By.XPATH, f'//*[@id="{btn_id}"]/ol/a[1]/li/button').text)
        driver.find_element(By.XPATH, f'//*[@id="{btn_id}"]/ol/a[1]/li/button').send_keys(Keys.ENTER)
        loading_check(curr_page)
    elif curr_page%10 == 0 :
        # 20, 30, ... 페이지일때 다음버튼 클릭
        # print("222 next : ", driver.find_element(By.XPATH, f'//*[@id="{btn_id}"]/ol/a[3]/li/button').text)
        driver.find_element(By.XPATH, f'//*[@id="{btn_id}"]/ol/a[3]/li/button').send_keys(Keys.ENTER)
        loading_check(curr_page)
    else:
        # print("333 next : ", driver.find_element(By.XPATH, f'//*[@id="{btn_id}"]/ol/li[{(curr_page%10)+1}]').text)
        # driver.find_element(By.XPATH, f'//*[@id="{btn_id}"]/ol/li[{(curr_page%10)+1}]').click()
        driver.find_element(By.XPATH, f'//*[@id="{btn_id}"]/ol/li[{(curr_page%10)+1}]/button').send_keys(Keys.ENTER)
        loading_check(curr_page)

In [4]:
def get_subdata():
    
    print("    --- 보조사업목록 조회 start ", datetime.datetime.now(), " ---")  
    
    sub_data = []
    search_cnt2 = int(driver.find_element(By.XPATH, f'//*[@id="IH001002Q_searchCnt2"]').text)
    
    # 조회된 보조사업 목록이 있음
    if search_cnt2 != 0:                
            
        total_page2 = driver.find_element(By.XPATH, f'//*[@id="IH001002Q_totalPage2"]').text.split('/')
        total_page2 = list(map(int, total_page2))
        
        # 조회된 보조사업 페이지수만큼 보조사업내용 가져오기
        for currPage_sub in range(1, total_page2[1]+1):
            
            # 현재보조사업내용 가져오기
            soup_sub = BeautifulSoup(driver.page_source, 'html.parser')
            tbl_sub = soup_sub.select('#SBHE_DATAGRID_WHOLE_TABLE_IH001002QSubGridObj')
            parse_sub = parser.make2d(tbl_sub[0])

            sub_data = sub_data + parse_sub[3:]
            
            # 페이지 이동
            move_page(currPage_sub, total_page2[1], "sbGridPaging2")
            
            if currPage_sub == total_page2[1] and len(sub_data) == search_cnt2:
                print("    --- 보조사업목록 조회 finish ", datetime.datetime.now(), " ---") 
    else: 
        sub_data = [['', '', '', '', '', '', '', '']]
        print("    --- 보조사업목록 조회 finish ",sub_data, " ", datetime.datetime.now(), " ---")
    
    return sub_data

### 1. 페이지 불러오기

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('headless') # 브라우저 창을 띄우지 않음
# options.add_argument('window_size=1920x1080')
# options.add_argument('--start-maximized')
options.add_argument('disable-gpu')

# page 불러오기
driver = webdriver.Chrome('chromedriver.exe', chrome_options=options)
driver.implicitly_wait(time_to_wait=60) # get할때만 동작, click에서는 안됨

print("page load start ", datetime.datetime.now())
driver.get(url="https://opn.gosims.go.kr/opn/ih/ih001/getIH001002QView.do") 
print("page load finish ", datetime.datetime.now()) # implicitly_wait 확인이 끝날때까지 아래 함수는 동작하지 않음

D:\Project\crawling\.crawling\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  
D:\Project\crawling\.crawling\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


page load start  2022-02-11 18:08:53.706238
page load finish  2022-02-11 18:08:58.686191


### 2. 메인사업 검색

#### ↓↓↓ 검색조건 입력 ↓↓↓ 

In [6]:
save_dir = "./2020" #저장할 디렉토리

start_dt = "2020" # 검색 시작 회계연도
end_dt = "2020" # 검색 종료 회계연도
jrsdCode = "문화재청" # 중앙부처

In [7]:
start_page = 1

driver.find_element(By.XPATH, f'//*[@id="IH001002QFrmSrch_fromFsyr"]/option[text()="{start_dt}"]').click()
loading_check()

driver.find_element(By.XPATH, f'//*[@id="IH001002QFrmSrch_toFsyr"]/option[text()="{end_dt}"]').click()
loading_check()

driver.find_element(By.XPATH, f'//*[@id="IH001002QFrmSrch_jrsdCode"]/option[text()="{jrsdCode}"]').click()
driver.find_element(By.XPATH, f'//*[@id="btnRetrieveAsbzDdtlbzStdCsts"]').send_keys(Keys.ENTER)
loading_check()

print(driver.find_element(By.XPATH, f'//*[@id="IH001002Q_searchCnt"]').text)
print(driver.find_element(By.XPATH, f'//*[@id="IH001002Q_totalPage"]').text)
print(driver.find_element(By.XPATH, f'//*[@id="SBHE_DATAGRID_WHOLE_TBODY_IH001002QMainGridObj"]/tr[2]').text.split('\n')) # 2번부터
# print(driver.find_element(By.XPATH, f'//*[@id="SBHE_IH001002QMainGridObj_1_2"]').text) # 1번부터

===>     [<div class="loadingMask on"><ul><li class="img1">로딩 중</li><li class="img2">로딩 중</li><li class="img3 on">로딩 중</li></ul></div>]
===>     []
===>     []
===>     []
101
1/11
['1', '2020', '[0006] IoT시스템(침입경보시스템)지원', '[330-01] 자치단체경상보조', '1,320,000', '문화재청']


### 2.1) ↓↓↓ 에러가 나서 다시 시작해야할 경우만 아래의 내용을 주석을 풀어서 수행한다. ↓↓↓
ElementClickInterceptedException

하던 중 에러가 날 경우, 해당 페이지까지 수동으로 이동을 해주어함

ex)
80번 페이지까지 csv파일이 생성되고 에러가 났다면  
81번 페이지부터 시작해야하므로, 81번페이지까지 이동을 해주어야 한다.

In [8]:
# start_page = 81

# for currPage in range(1, start_page+1):
#     # 메인화면 : sbGridPaging, 서브화면 : sbGridPaging2
#     move_page(currPage, start_page, "sbGridPaging", "restart")   

####   ↑↑↑ 에러가 나서 다시 시작해야할 경우만 수행

In [9]:
# 현재 위치 확인
driver.find_element(By.XPATH, f'//*[@id="IH001002Q_totalPage"]').text.split('/')

['1', '11']

### 3. 메인목록 별 보조내역 수집

In [10]:
main_columns = ["순번", "회계연도", "내역사업", "지출세목", "예산현액", "중앙부처"]
sub_columns = ["순번", "보조사업", "보조사업차수", "보조사업자", "(사업비)국고보조금(단위:천원)", "(사업비)지자체부담금(단위:천원)", "(사업비)자기부담금(단위:천원)", "(사업비)기타부담금(단위:천원)", "(사업비)합계(단위:천원)"]

search_cnt = int(driver.find_element(By.XPATH, f'//*[@id="IH001002Q_searchCnt"]').text)
start_time = datetime.datetime.now()
print("데이터 수집 start ", start_time)  

데이터 수집 start  2022-02-11 18:12:47.204711


In [11]:
# 조회된 메인사업 목록이 있음
if search_cnt != 0:
    
    totalPage = driver.find_element(By.XPATH, f'//*[@id="IH001002Q_totalPage"]').text.split('/')
    totalPage = list(map(int, totalPage))
    
    # 조회된 메인사업 페이지수만큼 메인내용 처리
    for currPage in range(start_page, totalPage[1]+1):
        clear_output(wait=True)
        print("<< ", currPage, " >>")   
        
        currPage_data = []
        
        # 한 페이지에 있는 목록 갯수 구하기
        curr_tbl = driver.find_element(By.XPATH, f'//*[@id="SBHE_DATAGRID_WHOLE_TBODY_IH001002QMainGridObj"]')
        curr_tbl_cnt = len(curr_tbl.find_elements(By.TAG_NAME, 'tr'))
        
        # 한 페이지에 있는 목록개수만큼 보조내역 읽기
        for curr_row in range(1, curr_tbl_cnt):            
            driver.find_element(By.XPATH, f'//*[@id="SBHE_IH001002QMainGridObj_{curr_row}_2"]').click() # 현재 사업을 클릭
            loading_check()

            curr_main_data = [driver.find_element(By.XPATH, f'//*[@id="SBHE_DATAGRID_WHOLE_TBODY_IH001002QMainGridObj"]/tr[{curr_row+1}]').text.split('\n')]
            print(curr_main_data)
            
            curr_sub_data = get_subdata()
            
            curr_row_merge = list(map(list.__add__, curr_main_data*len(curr_sub_data), curr_sub_data))
            currPage_data = currPage_data + curr_row_merge
            
        currPage_df = pd.DataFrame(data = currPage_data, columns= main_columns+sub_columns)
            
        save_path = f'{save_dir}/{start_dt}_{end_dt}_{jrsdCode}_{currPage:0>3}.csv'

        currPage_df.to_csv(save_path, index=False, encoding='utf-8-sig')       
        
         # 페이지 이동
        move_page(currPage, totalPage[1], "sbGridPaging")

print("데이터 수집 finish ", datetime.datetime.now(), " 소요시간 : ", datetime.datetime.now() - start_time)  

<<  11  >>
===>     []
[['101', '2020', '[0009] 향교서원문화재 활용사업(전국)', '[330-01] 자치단체경상보조', '5,068,000', '문화재청']]
    --- 보조사업목록 조회 start  2022-02-11 18:24:33.322762  ---
===>  1   []
===>  2   []
===>  3   []
===>  4   []
===>  5   []
===>  6   []
===>  7   []
===>  8   []
===>  9   []
===>  10   []
===>  11   []
===>  12   []
===>  13   []
===>  14   []
===>  15   []
===>  16   []
===>  17   []
===>  18   []
===>  19   []
===>  20   []
===>  21   []
      22  서브의 마지막 페이지 입니다.
    --- 보조사업목록 조회 finish  2022-02-11 18:25:07.085291  ---
메인의 마지막 페이지 입니다.
데이터 수집 finish  2022-02-11 18:25:07.091887  소요시간 :  0:12:19.887176
